In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
from pathlib import Path
import numpy as np
dataset_path = Path("/data/dzeiberg/mave_calibration/data/Boettcher_TP53/")
import sys
sys.path.append("..")

AUTHOR_TRANSCRIPT = ""

In [2]:
data = pd.read_excel(dataset_path / "raw"/ "TP53-MITEseq_collapsed_aa_fromIntendedCodons_inFraction_BoettcherSteffen.xlsx")
data = data.assign(author_transcript = AUTHOR_TRANSCRIPT)

In [3]:
data.head()

,Unnamed: 0,POS,Wt_codon,Wt_aa,Vt_aa,R1_Nutlin_GFP_high,R1_Nutlin_GFP_low,R1_DMSO_GFP_high,R1_DMSO_GFP_low,R3_Nutlin_GFP_high,R3_Nutlin_GFP_low,R3_DMSO_GFP_high,R3_DMSO_GFP_low,pDNAPool_PCR,pUC57-p53Clonal_straight,group,R1_Nutlin_Ratio_Lo_Hi,R3_Nutlin_Ratio_Lo_Hi,R1_DMSO_Ratio_Lo_Hi,R3_DMSO_Ratio_Lo_Hi,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,author_transcript
0,1,2,GAG,E,N,0.002163,0.002726,0.002788,0.002508,0.002449,0.002279,0.002827,0.001957,0.003390,0.000000,Intended_VT,0.100504,-0.031149,-0.045897,-0.159793,NaN,NaN,NaN,0.131653,0.113896,
1,2,2,GAG,E,K,0.001408,0.001317,0.001543,0.001076,0.001365,0.001464,0.001317,0.001706,0.001262,0.000025,Intended_VT,-0.028912,0.030377,-0.156697,0.112438,NaN,NaN,NaN,NaN,NaN,
2,3,2,GAG,E,T,0.001125,0.000847,0.001012,0.000842,0.000931,0.001098,0.000992,0.000727,0.001334,0.000000,Intended_VT,-0.123009,0.071713,-0.079905,-0.135260,NaN,NaN,NaN,NaN,NaN,
3,4,2,GAG,E,S,0.000714,0.000905,0.000902,0.001035,0.000743,0.000880,0.000893,0.000975,0.001124,0.000000,Intended_VT,0.102814,0.073928,0.059944,0.038489,NaN,NaN,NaN,NaN,NaN,
4,5,2,GAG,E,I,0.001009,0.001301,0.001087,0.000938,0.000786,0.000943,0.000916,0.000468,0.001192,0.000000,Intended_VT,0.110494,0.078855,-0.064186,-0.291672,NaN,NaN,NaN,NaN,NaN,


In [4]:
from Bio.PDB.Polypeptide import protein_letters_3to1
protein_letters_1to3 = {v:k.title() for k,v in protein_letters_3to1.items()}
protein_letters_1to3['Z'] = 'Glx'
protein_letters_1to3['B'] = 'Asx'
def make_hgvs_pro(r):
    return f"p.{protein_letters_1to3[r['Wt_aa']]}{r['POS']}{protein_letters_1to3[r['Vt_aa']]}"

In [5]:
data = data.assign(hgvs_pro = data.apply(make_hgvs_pro, axis=1))

drop ambiguous amino acids

In [6]:
data = data[~((data.hgvs_pro.str.contains("Glx") | (data.hgvs_pro.str.contains("Asx"))))]

In [7]:
data

,Unnamed: 0,POS,Wt_codon,Wt_aa,Vt_aa,R1_Nutlin_GFP_high,R1_Nutlin_GFP_low,R1_DMSO_GFP_high,R1_DMSO_GFP_low,R3_Nutlin_GFP_high,R3_Nutlin_GFP_low,R3_DMSO_GFP_high,R3_DMSO_GFP_low,pDNAPool_PCR,pUC57-p53Clonal_straight,group,R1_Nutlin_Ratio_Lo_Hi,R3_Nutlin_Ratio_Lo_Hi,R1_DMSO_Ratio_Lo_Hi,R3_DMSO_Ratio_Lo_Hi,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,author_transcript,hgvs_pro
0,1,2,GAG,E,N,0.002163,0.002726,0.002788,0.002508,0.002449,0.002279,0.002827,0.001957,0.003390,0.000000,Intended_VT,0.100504,-0.031149,-0.045897,-0.159793,NaN,NaN,NaN,0.131653,0.113896,,p.Glu2Asn
1,2,2,GAG,E,K,0.001408,0.001317,0.001543,0.001076,0.001365,0.001464,0.001317,0.001706,0.001262,0.000025,Intended_VT,-0.028912,0.030377,-0.156697,0.112438,NaN,NaN,NaN,NaN,NaN,,p.Glu2Lys
2,3,2,GAG,E,T,0.001125,0.000847,0.001012,0.000842,0.000931,0.001098,0.000992,0.000727,0.001334,0.000000,Intended_VT,-0.123009,0.071713,-0.079905,-0.135260,NaN,NaN,NaN,NaN,NaN,,p.Glu2Thr
3,4,2,GAG,E,S,0.000714,0.000905,0.000902,0.001035,0.000743,0.000880,0.000893,0.000975,0.001124,0.000000,Intended_VT,0.102814,0.073928,0.059944,0.038489,NaN,NaN,NaN,NaN,NaN,,p.Glu2Ser
4,5,2,GAG,E,I,0.001009,0.001301,0.001087,0.000938,0.000786,0.000943,0.000916,0.000468,0.001192,0.000000,Intended_VT,0.110494,0.078855,-0.064186,-0.291672,NaN,NaN,NaN,NaN,NaN,,p.Glu2Ile
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7869,7870,393,GAC,D,V,0.000135,0.000078,0.000165,0.000052,0.000143,0.000066,0.000138,0.000116,0.000179,0.000000,Intended_VT,-0.237826,-0.334245,-0.499130,-0.073448,NaN,NaN,NaN,NaN,NaN,,p.Asp393Val
7870,7871,393,GAC,D,Y,0.000153,0.000165,0.000235,0.000327,0.000268,0.000142,0.000460,0.000179,0.000558,0.000000,Intended_VT,0.032115,-0.274187,0.143179,-0.409503,NaN,NaN,NaN,NaN,NaN,,p.Asp393Tyr
7872,7873,393,GAC,D,W,0.000069,0.000054,0.000114,0.000045,0.000080,0.000112,0.000147,0.000193,0.000161,0.000000,Intended_VT,-0.109980,0.146388,-0.402910,0.119189,NaN,NaN,NaN,NaN,NaN,,p.Asp393Trp
7873,7874,393,GAC,D,C,0.000089,0.000044,0.000198,0.000047,0.000155,0.000015,0.000123,0.000033,0.000179,0.000000,Intended_VT,-0.310890,-1.010345,-0.626567,-0.569321,NaN,NaN,NaN,NaN,NaN,,p.Asp393Cys


use average of R1_Nutlin_Ratio_Lo_Hi and R3_Nutlin_Ratio_Lo_Hi as the score

In [8]:
data = data.assign(score=data.loc[:,['R1_Nutlin_Ratio_Lo_Hi', 'R3_Nutlin_Ratio_Lo_Hi']].apply(np.mean, axis=1))

In [9]:
data

,Unnamed: 0,POS,Wt_codon,Wt_aa,Vt_aa,R1_Nutlin_GFP_high,R1_Nutlin_GFP_low,R1_DMSO_GFP_high,R1_DMSO_GFP_low,R3_Nutlin_GFP_high,R3_Nutlin_GFP_low,R3_DMSO_GFP_high,R3_DMSO_GFP_low,pDNAPool_PCR,pUC57-p53Clonal_straight,group,R1_Nutlin_Ratio_Lo_Hi,R3_Nutlin_Ratio_Lo_Hi,R1_DMSO_Ratio_Lo_Hi,R3_DMSO_Ratio_Lo_Hi,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,author_transcript,hgvs_pro,score
0,1,2,GAG,E,N,0.002163,0.002726,0.002788,0.002508,0.002449,0.002279,0.002827,0.001957,0.003390,0.000000,Intended_VT,0.100504,-0.031149,-0.045897,-0.159793,NaN,NaN,NaN,0.131653,0.113896,,p.Glu2Asn,0.034678
1,2,2,GAG,E,K,0.001408,0.001317,0.001543,0.001076,0.001365,0.001464,0.001317,0.001706,0.001262,0.000025,Intended_VT,-0.028912,0.030377,-0.156697,0.112438,NaN,NaN,NaN,NaN,NaN,,p.Glu2Lys,0.000733
2,3,2,GAG,E,T,0.001125,0.000847,0.001012,0.000842,0.000931,0.001098,0.000992,0.000727,0.001334,0.000000,Intended_VT,-0.123009,0.071713,-0.079905,-0.135260,NaN,NaN,NaN,NaN,NaN,,p.Glu2Thr,-0.025648
3,4,2,GAG,E,S,0.000714,0.000905,0.000902,0.001035,0.000743,0.000880,0.000893,0.000975,0.001124,0.000000,Intended_VT,0.102814,0.073928,0.059944,0.038489,NaN,NaN,NaN,NaN,NaN,,p.Glu2Ser,0.088371
4,5,2,GAG,E,I,0.001009,0.001301,0.001087,0.000938,0.000786,0.000943,0.000916,0.000468,0.001192,0.000000,Intended_VT,0.110494,0.078855,-0.064186,-0.291672,NaN,NaN,NaN,NaN,NaN,,p.Glu2Ile,0.094674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7869,7870,393,GAC,D,V,0.000135,0.000078,0.000165,0.000052,0.000143,0.000066,0.000138,0.000116,0.000179,0.000000,Intended_VT,-0.237826,-0.334245,-0.499130,-0.073448,NaN,NaN,NaN,NaN,NaN,,p.Asp393Val,-0.286036
7870,7871,393,GAC,D,Y,0.000153,0.000165,0.000235,0.000327,0.000268,0.000142,0.000460,0.000179,0.000558,0.000000,Intended_VT,0.032115,-0.274187,0.143179,-0.409503,NaN,NaN,NaN,NaN,NaN,,p.Asp393Tyr,-0.121036
7872,7873,393,GAC,D,W,0.000069,0.000054,0.000114,0.000045,0.000080,0.000112,0.000147,0.000193,0.000161,0.000000,Intended_VT,-0.109980,0.146388,-0.402910,0.119189,NaN,NaN,NaN,NaN,NaN,,p.Asp393Trp,0.018204
7873,7874,393,GAC,D,C,0.000089,0.000044,0.000198,0.000047,0.000155,0.000015,0.000123,0.000033,0.000179,0.000000,Intended_VT,-0.310890,-1.010345,-0.626567,-0.569321,NaN,NaN,NaN,NaN,NaN,,p.Asp393Cys,-0.660617


In [10]:
data.to_csv(dataset_path / 'scoreset.csv', index=False)